Example stadincijfers usage in python
=========

Make sure that stadincijfers is installed.
If needed, run the following in terminal:

    pip install stadincijfers


In [1]:
import pandas as pd

In [2]:
from stadincijfers import stadincijfers

In [3]:
sic = stadincijfers("antwerpen")

### Geolevels and periodlevels.

You can get a dictionary of available periodlevels and geolevels on the server by calling `.geolevels()` or `.periodlevels()` on the sic-object. 

In [4]:
sic.periodlevels()

{'halfyear': 'Halfjaar',
 'month': 'Maand',
 'quarter': 'Kwartaal',
 'year': 'Jaar'}

### Getting the lists of available variables

You can also get the open available variables (cijfers) and their name, but this can take a while to read them all. These servers seem to have a lot of data. 

You need to give a starting en endpoint for this tool to loop through, they will be fetched in blocks of 10. Something like `sic.odataVariables(0, 100000)` will get them all.

Best to save your results to a file, so you don't need to fetch them every time. 

In [6]:
vars = sic.odataVariables(16700, 17000)
vars_s = pd.Series(vars)
vars_s.to_csv("data/vars.csv")
vars_s.tail()

reading data, lines 16700 to 17000 this can take a while
16700 16710 16720 16730 16740 16750 16760 16770 16780 16790 16800 16810 16820 16830 16840 16850 16860 16870 16880 16890 16900 16910 16920 16930 16940 16950 16960 16970 16980 16990 17000 

sm_mo_ws_elektrfiets_sum    Verplaatsing werk/school: vaak met elektrische...
sm_mo_ws_fiets                       Verplaatsing werk/school: vaak met fiets
sm_mo_ws_fiets_clt               Verplaatsing werk/school: vaak met fiets_CLT
sm_mo_ws_fiets_clt_og         Verplaatsing werk/school: vaak met fiets_CLT_OG
sm_mo_ws_fiets_sum               Verplaatsing werk/school: vaak met fiets_SUM
dtype: object

### Getting the data 

If you have a variable from the odataVariables-list you want to explore, you can fetch them with `selectiontableasjson` as a json-object or `selectiontableasDataframe` as a pandas-dataframe. You also need to specify **geolevel** , **periodlevel** and which **period** (usually a year) you want to select data for. 

In [5]:
# get data as a json object convert to dict
v = sic.selectiontableasjson('autodeelplaatsen_cambio', geolevel='sector', periodlevel='year', period=2020 )
v['rows'][0:10]

[['12MQ', '2020', ''],
 ['A00-', '2020', '2'],
 ['A01-', '2020', ''],
 ['A02-', '2020', '2'],
 ['A03-', '2020', ''],
 ['A04-', '2020', '3'],
 ['A05-', '2020', ''],
 ['A081', '2020', ''],
 ['A10-', '2020', ''],
 ['A11-', '2020', '5']]

In [6]:
cambio2016 = sic.selectiontableasDataframe('autodeelplaatsen_cambio', geolevel='sector', periodlevel='year', period=2016)
cambio2016.head(10)

,Geo,Perioden,# autodeelplaatsen Cambio
0,12MQ,2016,NaN
1,A00-,2016,2.0
2,A01-,2016,NaN
3,A02-,2016,2.0
4,A03-,2016,NaN
5,A04-,2016,3.0
6,A05-,2016,NaN
7,A081,2016,NaN
8,A10-,2016,NaN
9,A11-,2016,5.0


For cube variables, you can specify dimension levels using **dimlevel**. To get the available levels, call `.dim_dict()` on the sic-object and specify the variable.

In [8]:
#get the available dimension levels for a specific variable
sic.dim_dict('dna_kubus1111_lft_gesl_wr')

{'dna_leeftijd': ['dna_leeftijdsklasse_1',
  'dna_leeftijdsklasse_10',
  'dna_leeftijdsklasse_20',
  'dna_leeftijdsklasse_5',
  'dna_leeftijdsklasse_5_95p',
  'dna_leeftijdsklasse_5max100',
  'dna_leeftijdsklasse_85p',
  'dna_leeftijdsklasse_actief',
  'dna_leeftijdsklasse_basis',
  'dna_leeftijdsklasse_dna',
  'dna_leeftijdsklasse_dummy50plus',
  'dna_leeftijdsklasse_ksz'],
 'dna_geslacht': ['dna_geslacht'],
 'dna_wachtregister': ['dna_wachtregister']}

In [11]:
#specify dimlevel (if more than one, separate them with a comma without spaces)
sic.selectiontableasDataframe('dna_kubus1111_lft_gesl_wr', geolevel='gemeente',
                              dimlevel='dna_leeftijdsklasse_actief,dna_geslacht')

,VariableDimension,Perioden,leeftijd,geslacht,"bevolking naar leeftijd, geslacht, wachtregister"
0,Antwerpen,2022,Minderjarigen (0-17 jaar),man,61350
1,Antwerpen,2022,Minderjarigen (0-17 jaar),vrouw,58888
2,Antwerpen,2022,Actieve leeftijd (18-64 jaar),man,168099
3,Antwerpen,2022,Actieve leeftijd (18-64 jaar),vrouw,157407
4,Antwerpen,2022,65-plussers,man,37939
5,Antwerpen,2022,65-plussers,vrouw,48816


### Doing calculations

In [7]:
cambioNu = sic.selectiontableasDataframe('autodeelplaatsen_cambio', geolevel='sector', periodlevel='year', period=2022)
"cambios staansplaatsen in 2016 {} en nu {}".format( cambio2016['# autodeelplaatsen Cambio'].sum() ,
                                            cambioNu['# autodeelplaatsen Cambio'].sum() ) 

'cambios staansplaatsen in 2016 115.0 en nu 176.0'

## Export to excel

In [4]:
from stadincijfers import stadincijfers
sic = stadincijfers('antwerpen')
bevdicht = sic.selectiontableasDataframe('bevdicht', geolevel='sector', periodlevel='year', period=2022)
bevdicht.to_excel('data\\bevdicht.xlsx')